# Lab 2 Sentiment Analysis

BYGB7977 Text Analytics 

Prepared by: Yilu Zhou, Yifan Ren, Ricardo Lu

## Content Summery
Today we are going to do sentiment analysis in 5 approaches, including:

2 dictionary approaches: 1. Bing Liu's Dictionary 2. LM Dictionary

3 python package approaches: 1. TextBlob 2. Vader 3. Vader in NLTK


# Load Dataset

In [1]:
import pandas as pd

In [4]:
# read file
df = pd.read_table("short_input.txt",header = None, names = ['Message'])
df.head()

,Message
0,0|0|5|The Canon Rebel T3i takes the consumer l...
1,"If you are considering the Rebel T3i vs T2i, t..."
2,Canon Rebel T3i vs. 60D vs. 7D
3,Sensor and Image Quality: All three cameras s...
4,Exposure Metering: The three cameras all share...


In [8]:
df['Message'][0]

'0|0|5|The Canon Rebel T3i takes the consumer level dSLR a couple steps closer to the mid-level Canon 60D with the addition of the rotating rear LCD screen, remote flash firing, and in-camera processing features.  The already highly competent, older Rebel T2i already shared many important features with the 60D (and even features of the semi-pro 7D) including the 18 MP sensor, 63-zone exposure metering system, high ISO performance, HD movie capabilities, and Digic 4 image processor.  With these new upgrades, it might make it even more difficult to choose between them.  But there are some important differences.'

In [9]:
data = df.Message.str.lower()

# Dictionary Approach

In [10]:
def count_pos_neg(data, positive_dict, negative_dict):
# count of positive and negative words that appeared in each message
# net count which is calculated by positive count subtracting negative count. 
    poscnt = []
    negcnt = []
    netcnt = []

    for nrow in range(0,len(data)):
        text = data[nrow]
        
        qa = 0
        qb = 0

        for word in positive_dict :
            if (word in text) :
                qa = qa + 1

        for word in negative_dict :
            if (word in text) :
                qb = qb + 1

        qc = qa - qb

        poscnt.append(qa)
        negcnt.append(qb)
        netcnt.append(qc)

    return (poscnt, negcnt, netcnt)

## Bing Liu's Dictionary

In [11]:
#run this line if you never download it before
import nltk
nltk.download("opinion_lexicon")

[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     /Users/nicolasclarke/nltk_data...
[nltk_data]   Unzipping corpora/opinion_lexicon.zip.


True

In [12]:
#import Bing Liu's dictionary
from nltk.corpus import opinion_lexicon

In [13]:
pos_list_BL=set(opinion_lexicon.positive())
neg_list_BL=set(opinion_lexicon.negative())

In [14]:
df['poscnt_BL'], df['negcnt_BL'], df['netcnt_BL'] = count_pos_neg(data, pos_list_BL, neg_list_BL)

In [15]:
df[['Message','poscnt_BL','negcnt_BL','netcnt_BL']].head(5)

,Message,poscnt_BL,negcnt_BL,netcnt_BL
0,0|0|5|The Canon Rebel T3i takes the consumer l...,3,8,-5
1,"If you are considering the Rebel T3i vs T2i, t...",8,12,-4
2,Canon Rebel T3i vs. 60D vs. 7D,0,0,0
3,Sensor and Image Quality: All three cameras s...,1,0,1
4,Exposure Metering: The three cameras all share...,3,7,-4


## LM Dictionary

In [16]:
def read_local_dictionary(file):
    # create dictionary list
    words_dict = []
    with open(file, "r") as f: 
        for line in f:
            t = line.strip().lower()
            words_dict.append(t)
    return words_dict

In [17]:
pos_list_LM = read_local_dictionary('positive-words-LM.txt')
neg_list_LM = read_local_dictionary('negative-words-LM.txt')

In [18]:
df['poscnt_LM'], df['negcnt_LM'], df['netcnt_LM'] = count_pos_neg(data, pos_list_LM, neg_list_LM)

In [19]:
df[['Message','poscnt_LM','negcnt_LM','netcnt_LM']].head(5)

,Message,poscnt_LM,negcnt_LM,netcnt_LM
0,0|0|5|The Canon Rebel T3i takes the consumer l...,0,3,-3
1,"If you are considering the Rebel T3i vs T2i, t...",1,5,-4
2,Canon Rebel T3i vs. 60D vs. 7D,0,0,0
3,Sensor and Image Quality: All three cameras s...,1,1,0
4,Exposure Metering: The three cameras all share...,2,7,-5


# Package Dictionay

## TextBlob

In [20]:
#run this line if you never download it before
!pip install textblob

     |████████████████████████████████| 636 kB 4.0 MB/s eta 0:00:01


In [21]:
from textblob import TextBlob

In [28]:
df["score_TextBlob"] = df["Message"].map(lambda x:TextBlob(x).sentiment.polarity)
#can switch polarity to subjectivity

TextBlob’s output for a polarity task is a float within the range [-1.0, 1.0] where -1.0 is a negative polarity and 1.0 is positive. This score can also be equal to 0, which stands for a neutral evaluation of a statement as it doesn’t contain any words from the training set.

In [29]:
df[["Message","score_TextBlob"]]

,Message,score_TextBlob
0,0|0|5|The Canon Rebel T3i takes the consumer l...,0.216303
1,"If you are considering the Rebel T3i vs T2i, t...",0.080702
2,Canon Rebel T3i vs. 60D vs. 7D,0.000000
3,Sensor and Image Quality: All three cameras s...,0.075000
4,Exposure Metering: The three cameras all share...,0.033333
...,...,...
466,"Leaving it in auto mode, even a child can use ...",0.130360
467,The battery life is very good. I purchased two...,0.420000
468,"0|114|5|I've always considered myself a ""Good ...",0.179560
469,0|115|5|I have had my Rebel Ti3 camera and kit...,0.255767


## Vader
### (Valence Aware Dictionary and sEntiment Reasoner)

In [31]:
#run this line if you never download it before
!pip install vadersentiment

In [32]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [33]:
analyzer = SentimentIntensityAnalyzer()
scores = [analyzer.polarity_scores(sentence) for sentence in data]

The compound score is computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive). This is the most useful metric if you want a single unidimensional measure of sentiment for a given sentence. Calling it a 'normalized, weighted composite score' is accurate.

It is also useful for researchers who would like to set standardized thresholds for classifying sentences as either positive, neutral, or negative. Typical threshold values (used in the literature cited on this page) are:

positive sentiment: compound score >= 0.05
neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
negative sentiment: compound score <= -0.05
The pos, neu, and neg scores are ratios for proportions of text that fall in each category (so these should all add up to be 1... or close to it with float operation). These are the most useful metrics if you want multidimensional measures of sentiment for a given sentence.

In [35]:
#scores

In [36]:
neg_s = [i["neg"] for i in scores]
neu_s = [i["neu"] for i in scores]
pos_s = [i["pos"] for i in scores]
compound_s = [i["compound"] for i in scores]

In [37]:
df['negscore_Vader'], df['neuscore_Vader'], df['posscore_Vader'], df['compound_Vader'] = neg_s, neu_s, pos_s, compound_s

In [38]:
df[['Message','negscore_Vader','neuscore_Vader','posscore_Vader','compound_Vader']].head(5)

,Message,negscore_Vader,neuscore_Vader,posscore_Vader,compound_Vader
0,0|0|5|The Canon Rebel T3i takes the consumer l...,0.062,0.869,0.069,0.1945
1,"If you are considering the Rebel T3i vs T2i, t...",0.041,0.841,0.119,0.9803
2,Canon Rebel T3i vs. 60D vs. 7D,0.211,0.789,0.000,-0.1531
3,Sensor and Image Quality: All three cameras s...,0.000,0.866,0.134,0.5859
4,Exposure Metering: The three cameras all share...,0.043,0.898,0.060,0.2831


## NLTK

The pretrained sentiment analyzer within NLTK was called VADER (Valence Aware Dictionary and Sentiment Reasoner). Yes, it the same as what we just did! Here is to introduce another way to call this model through NLTK.

In [39]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/nicolasclarke/nltk_data...


True

In [40]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [41]:
sia = SentimentIntensityAnalyzer()

In [42]:
scores = [sia.polarity_scores(sentence) for sentence in data]

In [43]:
scores

[{'neg': 0.062, 'neu': 0.867, 'pos': 0.071, 'compound': 0.2263},
 {'neg': 0.042, 'neu': 0.844, 'pos': 0.113, 'compound': 0.9736},
 {'neg': 0.211, 'neu': 0.789, 'pos': 0.0, 'compound': -0.1531},
 {'neg': 0.0, 'neu': 0.862, 'pos': 0.138, 'compound': 0.5859},
 {'neg': 0.043, 'neu': 0.896, 'pos': 0.06, 'compound': 0.2831},
 {'neg': 0.01, 'neu': 0.94, 'pos': 0.049, 'compound': 0.7279},
 {'neg': 0.0, 'neu': 0.849, 'pos': 0.151, 'compound': 0.9525},
 {'neg': 0.0, 'neu': 0.938, 'pos': 0.062, 'compound': 0.5267},
 {'neg': 0.0, 'neu': 0.941, 'pos': 0.059, 'compound': 0.8641},
 {'neg': 0.0, 'neu': 0.916, 'pos': 0.084, 'compound': 0.7351},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.077, 'neu': 0.821, 'pos': 0.101, 'compound': 0.128},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 0.87, 'pos': 0.13, 'compound': 0.2732},
 {'neg': 0.0, 'neu': 0.83, 'pos': 0.17, 'compound': 0.8402},
 {'neg': 0

In [44]:
neg_s_n = [i["neg"] for i in scores]
neu_s_n = [i["neu"] for i in scores]
pos_s_n = [i["pos"] for i in scores]
compound_s_n = [i["compound"] for i in scores]

In [45]:
df['negscore_NLTK'], df['neuscore_NLTK'], df['posscore_NLTK'], df['compound_NLTK'] = neg_s_n, neu_s_n, pos_s_n, compound_s_n

In [46]:
df[['Message','negscore_NLTK','neuscore_NLTK','posscore_NLTK','compound_NLTK']].head(5)

,Message,negscore_NLTK,neuscore_NLTK,posscore_NLTK,compound_NLTK
0,0|0|5|The Canon Rebel T3i takes the consumer l...,0.062,0.867,0.071,0.2263
1,"If you are considering the Rebel T3i vs T2i, t...",0.042,0.844,0.113,0.9736
2,Canon Rebel T3i vs. 60D vs. 7D,0.211,0.789,0.000,-0.1531
3,Sensor and Image Quality: All three cameras s...,0.000,0.862,0.138,0.5859
4,Exposure Metering: The three cameras all share...,0.043,0.896,0.060,0.2831
